# Requirements

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Data

In [4]:
for file in os.listdir(r'tennis_data/tennis_csv'):
    df_name = file.split('.')[0]
    globals()[df_name] = pd.read_csv(r'tennis_data/tennis_csv/' + file)

# Creative questions

### 1. The least risky matchs to bet on
### 2. Which physical factor has the greatest impact on the service score
### 3. Calculate the win-loss ratio for each player
### 4. How did the winners of the game get the most points?
### 5. Does being from the same country as the player and the tournament affect the player's win?
### 6. assuming that votes table are fan's predictions it has affect the player's win ?
### 7. what is the corrolation between votes and bet on players
### 8. wich seed has most win ?
### 9. wich player got the most points ?
### 10. is any corrolation between first serve and winning ?
### 11. wich season has players with high rank ?
### 12. whch catagory of statitics has most affect to winning ?
### 13.* countries with most wins 
### 14.* countries with most players
### 15.* countries with most hight rank players

<hr style="border:8px solid gray">

In [12]:
match_win_month = df_event[['match_id', 'winner_code', 'start_datetime']]
match_win_month['start_datetime'] = pd.to_datetime(match_win_month['start_datetime'], unit='s')
match_win_month['month'] = match_win_month['start_datetime'].dt.month
match_win_month.drop(columns=['start_datetime'], inplace=True)
match_win_month.dropna().shape
match_h_player = df_home_team[['match_id', 'name']].drop_duplicates()
match_a_player = df_away_team[['match_id', 'name']].drop_duplicates()
win_h_player = pd.merge(match_win_month.loc[match_win_month['winner_code'] == 1], match_h_player, on='match_id')
win_h_player.drop_duplicates(subset = ['match_id'], keep = 'first', inplace = True)
win_h_player.dropna(inplace = True)
win_a_player = pd.merge(match_win_month.loc[match_win_month['winner_code'] == 2], match_a_player, on='match_id')
win_a_player.drop_duplicates(subset = ['match_id'], keep = 'first', inplace = True)
win_a_player.dropna(inplace = True)
winners_name = pd.concat([win_h_player, win_a_player], axis = 0, ignore_index = True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10552\1004059285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_win_month['start_datetime'] = pd.to_datetime(match_win_month['start_datetime'], unit='s')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10552\1004059285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_win_month['month'] = match_win_month['start_datetime'].dt.month
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10552\1004059285.py:4: SettingWithCopyWarning: 
A value is trying to be s

## 1. The least risky matchs to bet on

In [ ]:
df_odds.dropna(axis='rows', inplace=True)
df_odds.drop(columns='is_live', inplace=True)
df_odds.drop(columns='suspended', inplace=True)

In [13]:
df_odds['fractional_value'] = df_odds['fractional_value'].replace('1/', '1')
df_odds['fractional_value'] = df_odds['fractional_value'].replace('13/', '1/3')
df_odds['fractional_value'] = df_odds['fractional_value'].replace('4/', '3/4')

In [8]:
df_odds['fractional_value'] = df_odds['fractional_value'].apply(lambda x: eval(x) if '/' in x else float(x))

In [14]:
match_fraction_win = df_odds[['match_id','market_name', 'choice_name', 'fractional_value', 'winnig']]
match_wname_wcode = winners_name[['match_id', 'name', 'winner_code']].rename(columns={'name' : 'winner_name'})
wname_fraction = pd.merge(match_fraction_win, match_wname_wcode, on='match_id')

In [ ]:
wname_fraction.loc[wname_fraction['winnig'] == True]

,match_id,market_name,choice_name,fractional_value,winnig,winner_name,winner_code
1,11673958,full_time,2,0.830000,True,Tirante T.,2.0
3,11673958,first_set_winner,2,0.833333,True,Tirante T.,2.0
5,11673958,total_games_won,Under,0.909091,True,Tirante T.,2.0
6,11673962,full_time,1,0.500000,True,Tirante T.,1.0
8,11673963,full_time,1,0.360000,True,Heide G.,1.0
...,...,...,...,...,...,...,...
770,11703442,full_time,1,0.083333,True,Tessa L.,1.0
773,11703443,full_time,2,0.727273,True,Kalina V.,2.0
774,11703445,full_time,1,0.833333,True,Maute J.,1.0
776,11703470,full_time,1,0.666667,True,Grey S.,1.0


In [ ]:
wname_fraction.groupby('winner_name').agg({'fractional_value' : 'mean'})

,fractional_value
winner_name,
Agamenone F.,0.882626
Alcaraz C.,1.918089
Alves M.,8.002500
Amariei I.,4.252500
Ambrogi L.,0.907692
...,...
Zormann M.,3.775000
Zverev A.,0.987549
de Bakker T.,1.500000


<hr style="border:8px solid gray">

## 3. Calculate the win-loss ratio for each player

In [ ]:
loser_h_player = pd.merge(match_win_month.loc[match_win_month['winner_code'] == 2], match_h_player, on='match_id')
loser_h_player.drop_duplicates(subset = ['match_id'], keep = 'first', inplace = True)
loser_h_player.dropna(inplace = True)
loser_a_player = pd.merge(match_win_month.loc[match_win_month['winner_code'] == 1], match_a_player, on='match_id')
loser_a_player.drop_duplicates(subset = ['match_id'], keep = 'first', inplace = True)
loser_a_player.dropna(inplace = True)
loesers_nam = pd.concat([loser_h_player, loser_a_player], axis = 0, ignore_index = True)

In [ ]:
winners_name

,match_id,winner_code,month,name
0,11673962,1.0,10,Tirante T.
1,11673963,1.0,9,Heide G.
2,11674008,1.0,9,Shelbayh A.
3,11674032,1.0,10,Machač T.
4,11674034,1.0,9,Draper J.
...,...,...,...,...
497,11702232,2.0,10,Aguilar Cardozo J.
498,11702238,2.0,10,Derdoy F.
499,11702243,2.0,10,Mouilleron Salvo M.
500,11702248,2.0,10,Noel M.


In [ ]:
losers_name

,match_id,winner_code,month,name
0,11673958,2.0,9,Barrios Vera T.
1,11673993,2.0,10,Ajduković D.
2,11674000,2.0,9,Escobedo E.
3,11674011,2.0,10,Crawford O.
4,11674026,2.0,9,Gasquet R.
...,...,...,...,...
306,11702218,1.0,10,Oliveira C.
307,11702221,1.0,10,Dedura-Palomero M.
308,11702244,1.0,10,Simonit E.
309,11703445,1.0,10,Nad P.


In [ ]:
win_count = winners_name.groupby('name', as_index=False).agg({'match_id' : 'count'}).sort_values(by=['match_id'], ascending=False).rename(columns={'match_id' : 'win_count'})

In [ ]:

winners_name['name'].duplicated().sum(), losers_name['name'].duplicated().sum()

(130, 0)

<hr style="border:8px solid gray">

## 5. Does being from the same country as the player and the tournament affect the player's win?

 <span style="color:#728C69 ; font-family:Times new roman ; font-size: 2em ;">extract country of players</span>

In [ ]:
home_player_contry = df_home_team[['match_id', 'name', 'birthplace']]
country = home_player_contry['birthplace'].str.split(',').str[-1]
home_player_contry = pd.concat([home_player_contry, country], axis=1)
home_player_contry.columns = ['match_id', 'name', 'birthplace', 'country']
home_player_contry.drop(columns='birthplace', inplace=True)
home_player_contry.dropna(inplace=True)

In [ ]:
away_player_contry = df_away_team[['match_id', 'name', 'birthplace']]
country = away_player_contry['birthplace'].str.split(',').str[-1]
away_player_contry = pd.concat([away_player_contry, country], axis=1)
away_player_contry.columns = ['match_id', 'name', 'birthplace', 'country']
away_player_contry.drop(columns='birthplace', inplace=True)
away_player_contry.dropna(inplace=True)

In [ ]:
players_country = pd.concat([away_player_contry, home_player_contry])

<span style="color:#728C69 ; font-family:Times new roman ; font-size: 2em ;">extraxt The country of the match</span>

In [ ]:
match_country = df_venue[['match_id', 'country']].dropna().rename(columns={'country' : 'match_country'})

<span style="color:#728C69 ; font-family:Times new roman ; font-size: 2em ;">compare birth country and match's country</span>

In [ ]:
countries = match_country.merge(players_country, on='match_id', how='left')

In [ ]:
winners_countries = countries.merge(winners_name[['match_id', 'name']], on='match_id')
winners_countries['match_country'].astype('str').iloc[15] == winners_countries['country'].astype('str').iloc[15]

False

In [ ]:
winners_countries['match_country'] = winners_countries['match_country'].str.strip()
winners_countries['country'] = winners_countries['country'].str.strip()

In [ ]:
same_countries = (winners_countries[(winners_countries['match_country'] == winners_countries['country'])
                                    & (winners_countries['name_x'] == winners_countries['name_y']) ].reset_index().drop(columns='index'))
home_country_winners = (same_countries.groupby('name_x').agg({'country' : 'count'})
                        .sort_values(by=['country'], ascending=False).reset_index().rename(columns={'name_x' : 'name'}))

In [ ]:
home_status = home_country_winners.merge(win_count[['name', 'win_count']], on='name')

In [ ]:
lose_count = losers_name.groupby('name').agg({'match_id' : 'count'}).rename(columns= {'match_id' : 'lose_count'})

In [ ]:
games = home_status.merge(lose_count, on='name', how='left')
games.fillna(0, inplace=True)
games['game_count'] = games['win_count'] + games['lose_count']